In [1]:
%matplotlib inline
import matplotlib.pyplot as plt
import h5py
from sklearn.model_selection import train_test_split
import numpy as np
import keras.backend as K
import keras
from keras.datasets import mnist
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten
from keras.layers import Conv2D, MaxPooling2D, Convolution2D, Activation
from keras import backend as K
from keras.optimizers import SGD
import os
os.environ["CUDA_VISIBLE_DEVICES"] = "0"
def load():
    f = h5py.File("data/dataset.h5")
    x = f['x'].value
    y = f['y'].value
    f.close()
    x_train , x_test, y_train, y_test = train_test_split(x,y,test_size=0.2,random_state=100)
    x_train = np.rollaxis(x_train, 1, 4)
    x_test = np.rollaxis(x_test, 1, 4)
    x_train = x_train  / 255.0
    x_test = x_test / 255.0
    return x_train, x_test, y_train, y_test

x_train, x_test, y_train, y_test = load()
print(y_train.shape)

def multitask_loss(y_true, y_pred):
    y_pred = K.clip(y_pred, K.epsilon(), 1 - K.epsilon())
    return K.mean(K.sum(- y_true * K.log(y_pred) - (1 - y_true) * K.log(1 - y_pred), axis=1))

Using TensorFlow backend.


(1600, 5)


In [2]:
batch_size = 100
num_classes = 5
epochs = 50


# input image dimensions
img_rows, img_cols = 100, 100
channels = 3

model = Sequential()
model.add(Conv2D(32, kernel_size=(3, 3),padding='same',input_shape=(img_rows, img_cols, channels)))
model.add(Activation('relu'))
model.add(Conv2D(32, (3, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))

model.add(Conv2D(64,(3, 3), padding='same'))
model.add(Activation('relu'))
model.add(Conv2D(64, (3, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))

model.add(Flatten())
model.add(Dense(512))
model.add(Activation('relu'))
model.add(Dropout(0.5))
model.add(Dense(num_classes))
model.add(Activation('sigmoid'))


model.compile(loss=multitask_loss,
              optimizer='adam',
              metrics=['accuracy'])

history = model.fit(x_train, y_train,
          batch_size=batch_size,
          epochs=epochs,
          verbose=1,
          validation_data=(x_test, y_test))

Train on 1600 samples, validate on 400 samples
Epoch 1/50
1600/1600 [==============================] - 5s 3ms/step - loss: 3.0101 - acc: 0.3031 - val_loss: 2.3952 - val_acc: 0.4900
Epoch 2/50
1600/1600 [==============================] - 1s 757us/step - loss: 2.1631 - acc: 0.4887 - val_loss: 2.0571 - val_acc: 0.5500
Epoch 3/50
1600/1600 [==============================] - 1s 759us/step - loss: 1.9264 - acc: 0.5763 - val_loss: 1.9910 - val_acc: 0.5775
Epoch 4/50
1600/1600 [==============================] - 1s 759us/step - loss: 1.8432 - acc: 0.5963 - val_loss: 1.9189 - val_acc: 0.6050
Epoch 5/50
1600/1600 [==============================] - 1s 758us/step - loss: 1.7358 - acc: 0.6231 - val_loss: 1.7934 - val_acc: 0.6275
Epoch 6/50
1600/1600 [==============================] - 1s 758us/step - loss: 1.6212 - acc: 0.6456 - val_loss: 1.7394 - val_acc: 0.6425
Epoch 7/50
1600/1600 [==============================] - 1s 764us/step - loss: 1.5187 - acc: 0.6556 - val_loss: 1.7011 - val_acc: 0.6425
Epo

In [3]:
def evaluate_acc(model):
    inference = []
    y_pred = model.predict(x_test)
    y_pred = (y_pred > 0.5) * 1.0
    
    for i in range(num_classes):
        acc = np.sum(y_pred[:,i] == y_test[:,i])/y_pred.shape[0]
        print(f'acc for task{i}={acc}')
evaluate_acc(model)

acc for task0=0.88
acc for task1=0.8375
acc for task2=0.8
acc for task3=0.9325
acc for task4=0.855


In [ ]:
def build_single_model():
    model = Sequential()
    model.add(Conv2D(32, kernel_size=(3, 3),padding='same',input_shape=(img_rows, img_cols, channels)))
    model.add(Activation('relu'))
    model.add(Conv2D(32, (3, 3)))
    model.add(Activation('relu'))
    model.add(MaxPooling2D(pool_size=(2, 2)))
    model.add(Dropout(0.25))

    model.add(Conv2D(64,(3, 3), padding='same'))
    model.add(Activation('relu'))
    model.add(Conv2D(64, (3, 3)))
    model.add(Activation('relu'))
    model.add(MaxPooling2D(pool_size=(2, 2)))
    model.add(Dropout(0.25))

    model.add(Flatten())
    model.add(Dense(512))
    model.add(Activation('relu'))
    model.add(Dropout(0.5))
    model.add(Dense(1))
    model.add(Activation('sigmoid'))
    model.compile(loss=multitask_loss,
              optimizer='adam',
              metrics=['accuracy'])
    return model

for i in range(5):
    model = build_single_model()
    y_use_train = y_train[:,i].reshape(-1, 1)
    y_use_test = y_test[:,i].reshape(-1, 1)
    history = model.fit(x_train, y_use_train,
              batch_size=batch_size,
              epochs=epochs,
              verbose=0,
              validation_data=(x_test, y_use_test))
    print(f'best val acc for task {i} is {np.max(history.history["val_acc"])}')

best val acc for task 0 is 0.9100000113248825
best val acc for task 1 is 0.8575000017881393
best val acc for task 2 is 0.8224999904632568


In [ ]:
x = np.zeros([2,3])
print(x[:,0].reshape(-1,1))